In [ ]:
# Cell 1: imports & basic config

import json
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict, Any
from dotenv import load_dotenv
import os

import numpy as np

# If you're using the OpenAI client:
from openai import OpenAI

# load_dotenv()  # looks for .env in current dir
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # or use env var

# Embedding model name (adjust to what you're actually using)
EMBEDDING_MODEL = "text-embedding-3-small"

# Chat model for intent parsing
INTENT_MODEL = "gpt-4.1-mini"  # or whatever you're on


In [3]:
# Cell 2: load YANG catalog

CATALOG_PATH = "sensor_catalog.jsonl"  # or .json

all_rows = []

with open(CATALOG_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        all_rows.append(json.loads(line))

len(all_rows)


54063

In [4]:
# Cell 3: define intent schema

@dataclass
class TelemetryIntent:
    # High-level task
    task: str                      # e.g. "generate_telemetry_config"

    # Protocol / domain
    protocol_tag: Optional[str]    # e.g. "bgp", "ospf", "qos"
    vendor: Optional[str]          # e.g. "cisco"
    os: Optional[str]              # e.g. "ios xr"

    # Transport / export details
    transport: Optional[str]       # e.g. "grpc"
    tls: Optional[bool]            # True / False / None
    destination_address: Optional[str]
    destination_port: Optional[int]

    # Telemetry sampling / style
    sample_interval_ms: Optional[int]  # if user mentions "every 30 seconds"

    # What kind of data they care about
    metric_focus: List[str]       # e.g. ["neighbors", "stats", "status"]
    extra_requirements: str       # free-text notes (e.g. "no tls", "self-describing-gpb")

    # Raw original query (so you can always fall back)
    original_query: str

    # Helper: construct from dict safely
    @staticmethod
    def from_dict(d: Dict[str, Any]) -> "TelemetryIntent":
        return TelemetryIntent(
            task=d.get("task", "generate_telemetry_config"),
            protocol_tag=d.get("protocol_tag"),
            vendor=d.get("vendor"),
            os=d.get("os"),
            transport=d.get("transport"),
            tls=d.get("tls"),
            destination_address=d.get("destination_address"),
            destination_port=d.get("destination_port"),
            sample_interval_ms=d.get("sample_interval_ms"),
            metric_focus=d.get("metric_focus") or [],
            extra_requirements=d.get("extra_requirements", ""),
            original_query=d.get("original_query", ""),
        )


In [5]:
# Cell 4: build prompt for the intent LLM

INTENT_SYSTEM_PROMPT = """
You are a network telemetry assistant. Your job is to convert a natural language request
into a structured JSON object describing what telemetry configuration is needed.

Follow these rules:
- Only output valid JSON, no comments, no explanations.
- Use the following keys exactly:

{
  "task": string,                     // e.g. "generate_telemetry_config"
  "protocol_tag": string or null,     // one of: bgp, ospf, isis, mpls, ldp, multicast, routing, interfaces, qos, acl, platform, tunnel, or null
  "vendor": string or null,           // e.g. "cisco"
  "os": string or null,               // e.g. "ios xr"
  "transport": string or null,        // e.g. "grpc"
  "tls": boolean or null,             // true, false, or null if not specified
  "destination_address": string or null,
  "destination_port": integer or null,
  "sample_interval_ms": integer or null,   // telemetry sampling interval in ms if mentioned
  "metric_focus": array of strings,   // short tags like "neighbors", "routes", "stats", "summary", "status"
  "extra_requirements": string,       // any additional constraints (formats, comments, etc.)
  "original_query": string            // echo the original user query
}

If the user doesn't specify something, set it to null (or [] for arrays) instead of guessing too much.
When inferring protocol_tag, map phrases like "border gateway protocol" to "bgp".
""".strip()


In [ ]:
# Cell 5: LLM call -> JSON -> TelemetryIntent

def parse_intent(query: str) -> TelemetryIntent:
    response = client.chat.completions.create(
        model=INTENT_MODEL,
        messages=[
            {"role": "system", "content": INTENT_SYSTEM_PROMPT},
            {"role": "user", "content": query},
        ],
        temperature=0.0,
    )

    content = response.choices[0].message.content
    # content should be pure JSON per instructions
    try:    # Ensure original_query is filled

        data = json.loads(content)
    except json.JSONDecodeError as e:
        raise ValueError(f"Model did not return valid JSON: {e}\n\n{content}")

    # Ensure original_query is filled
    if not data.get("original_query"):
        data["original_query"] = query

    return TelemetryIntent.from_dict(data)


In [8]:
# Cell 6: test with your sample query

query = (
    "generate telemetry configuration for cisco ios xr about bgp protocol ? "
    "Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. "
    "Choose relevant sensor-paths."
)

intent = parse_intent(query)
intent


TelemetryIntent(task='generate_telemetry_config', protocol_tag='bgp', vendor='cisco', os='ios xr', transport='grpc', tls=False, destination_address='192.0.2.0', destination_port=57500, sample_interval_ms=None, metric_focus=['neighbors', 'routes', 'summary', 'status'], extra_requirements='Use relevant BGP sensor-paths for Cisco IOS XR telemetry configuration.', original_query='generate telemetry configuration for cisco ios xr about bgp protocol ? Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. Choose relevant sensor-paths.')

In [9]:
# Cell 7: intent -> canonical text for embedding

def intent_to_search_text(intent: TelemetryIntent) -> str:
    parts = []

    parts.append(f"Task: {intent.task}")
    if intent.protocol_tag:
        parts.append(f"Protocol: {intent.protocol_tag}")
    if intent.vendor:
        parts.append(f"Vendor: {intent.vendor}")
    if intent.os:
        parts.append(f"OS: {intent.os}")

    if intent.transport:
        parts.append(f"Transport: {intent.transport}")
    if intent.tls is not None:
        tls_str = "no-tls" if intent.tls is False else "tls"
        parts.append(f"Security: {tls_str}")
    if intent.destination_address or intent.destination_port:
        parts.append(
            "Destination: "
            f"{intent.destination_address or ''}:{intent.destination_port or ''}"
        )
    if intent.sample_interval_ms:
        parts.append(f"Sample interval ms: {intent.sample_interval_ms}")

    if intent.metric_focus:
        parts.append("Metric focus: " + ", ".join(intent.metric_focus))

    if intent.extra_requirements:
        parts.append("Extra: " + intent.extra_requirements)

    # Optional: include the original natural language query at the end for extra semantics
    parts.append("Original query: " + intent.original_query)

    return "\n".join(parts)


In [12]:
# Cell 8: compute embeddings for all YANG entries

def embed_text(text: str) -> np.ndarray:
    resp = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=text,
    )
    vec = np.array(resp.data[0].embedding, dtype=np.float32)
    return vec

yang_vectors = []
yang_ids = []
yang_meta = []  # optional: keep references to the rows

for row in all_rows[:10]:
    vec = embed_text(row["search_text"])
    yang_vectors.append(vec)
    yang_ids.append(row["id"])
    yang_meta.append(row)

yang_matrix = np.vstack(yang_vectors)  # shape: (N, D)
yang_matrix.shape


(10, 1536)

In [13]:
# Cell 9: similarity search over YANG catalog

def cosine_similarity_matrix(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    # a: (1, D) or (D,)
    # b: (N, D)
    a = a.reshape(1, -1)
    a_norm = a / np.linalg.norm(a, axis=1, keepdims=True)
    b_norm = b / np.linalg.norm(b, axis=1, keepdims=True)
    return (a_norm @ b_norm.T)[0]  # shape (N,)

def find_top_k_paths(intent: TelemetryIntent, k: int = 10) -> List[Dict[str, Any]]:
    # 1) Optional: filter by protocol_tag before similarity
    filtered_indices = list(range(len(all_rows)))
    if intent.protocol_tag:
        filtered_indices = [
            i for i, row in enumerate(all_rows)
            if row.get("protocol_tag") == intent.protocol_tag
        ]
        if not filtered_indices:
            # fallback: no filter if nothing matches
            filtered_indices = list(range(len(all_rows)))

    # 2) Embed the intent text
    intent_text = intent_to_search_text(intent)
    intent_vec = embed_text(intent_text)

    # 3) Compute similarity only on filtered subset
    sub_matrix = yang_matrix[filtered_indices]  # (M, D)
    sims = cosine_similarity_matrix(intent_vec, sub_matrix)

    # 4) Take top-k within this subset
    top_k_idx = np.argsort(-sims)[:k]   # descending

    results = []
    for rank, idx_in_sub in enumerate(top_k_idx):
        global_idx = filtered_indices[idx_in_sub]
        row = all_rows[global_idx]
        score = float(sims[idx_in_sub])
        results.append({
            "rank": rank + 1,
            "score": score,
            "id": row["id"],
            "path": row["path"],
            "module": row["module"],
            "protocol_tag": row.get("protocol_tag"),
            "category": row.get("category"),
            "description": row.get("description"),
        })
    return results


In [14]:
# Cell 10: full pipeline test

query = (
    "generate telemetry configuration for cisco ios xr about bgp protocol ? "
    "Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. "
    "Choose relevant sensor-paths."
)

intent = parse_intent(query)
print("Parsed intent:", intent)

top_paths = find_top_k_paths(intent, k=5)
for r in top_paths:
    print(f"[{r['rank']}] score={r['score']:.3f}")
    print("  Module :", r["module"])
    print("  Path   :", r["path"])
    print("  Proto  :", r["protocol_tag"])
    print("  Tags   :", r["category"])
    print("  Desc   :", r["description"])
    print()


Parsed intent: TelemetryIntent(task='generate_telemetry_config', protocol_tag='bgp', vendor='cisco', os='ios xr', transport='grpc', tls=False, destination_address='192.0.2.0', destination_port=57500, sample_interval_ms=None, metric_focus=['neighbors', 'routes', 'summary', 'status'], extra_requirements='Use relevant BGP sensor-paths for Cisco IOS XR telemetry over gRPC without TLS.', original_query='generate telemetry configuration for cisco ios xr about bgp protocol ? Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. Choose relevant sensor-paths.')


IndexError: index 512 is out of bounds for axis 0 with size 10